### README

## `Analysis.py`

**What does the script do?**

* **Latents vs. footshock** – saves per-latent plots (`latentXX.png`/`.pdf`).  
* **Latents vs. speed** – overlays speed trace with latent states and keeps the footshocks.  
* **Latents vs. pupil** – aligns/cleans pupil, overlays per latent, pupil axis ≥ 0.  
* **Discrete states (two styles)** – exports `states_style1.png` and `states_style2.png` the two styles are only different in how they look if there are too litle state changes we make the changes more visible.  
* **Switch statistics** – computes count, rate (/min), occupancy; saves PNGs + CSV of stats.  
* **Mutual information (MI)** – per latent vs. speed/pupil; saves detailed CSVs and bar plots.  
* **MI pre/post shock** – bar chart + CSV with FDR/perm statistics.  
* **Vector fields** – plots rSLDS state-wise latent flow in PCA (2 ) plane.  
* **Forward Simulation Error (FSE)** – plots MSE vs. horizon curve + saves `FSE.npy`.  

---

**Inputs (expected on Drive)**  

* Model outputs per rat/variant: `x_hat.npy`, `z_hat.npy`, `footshock.npy`, optionally `speed.npy`, `pupil.npy`, and model pickle (`*model*.pkl`).  
* Area/Responsive CSVs (wide format, must include `time_s`).  
* HDF5 per rat for signals when not available in CSV (e.g., `NpxFiringRate_Behavior_SBL_10msBINS_0smoothing.hdf5`).  

---

**Where it writes**  

* `/content/drive/My Drive/rSLDS/All-Plots/latents_footshock/…`  
* `/latents_speed/…`, `/latents_pupil/…`, `/discrete_states_both/…`  
* `/switch_summary/…`, `/mi_speed/…`, `/mi_pupil/…`, `/mi_prepost/…`  
* `/vector_fields/…`, `/fse/…`  

---

**How to run (in Colab)**  

* Open the notebook or run the exported `analysis.py` in Colab.  
* Set lists: `RAT_IDS`, `MODEL_SUFFIXES`.  
* Check Drive paths at the top of each section.  
* Run only the cells/sections you need.  




This notebook analsis the results of the rSLDS model and saves the outputs on drive.

### Installations and Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!pip uninstall -y numpy scipy numba scikit-learn pandas matplotlib seaborn autograd ssm
!pip install "numpy==1.26.4" "scipy==1.10." "numba==0.58." "scikit-learn==1.3.*" \
            "pandas==1.5." "matplotlib==3.7." "seaborn==0.12.*"

In [ ]:
!git clone https://github.com/lindermanlab/ssm.git
!pip install git+https://github.com/lindermanlab/ssm.git --no-build-isolation

In [ ]:
!git clone https://github.com/ElifAyten/rSLDS.git

In [9]:
# =========================
# Standard library
# =========================
import os
import sys
import glob
import pickle
import traceback
from pathlib import Path

import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.ndimage import gaussian_filter1d
from scipy.interpolate import interp1d
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import ssm  # recurrent SLDS package

repo_root = Path("/content/rSLDS")
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

from rSLDS.load_dataset import load_rat_data
from rSLDS.pca import pca_summary, plot_pca_cumsum
from rSLDS.normalize_firing_rates import normalize_firing_rates
from rSLDS.match_data_with_metadata import match_units_to_hdf5
from rSLDS.population_acitivity_region import population_activity_by_region
from rSLDS.create_sub_data import export_area_splits
from rSLDS.create_sub_data_2 import export_responsive_tables
from rSLDS.modelling import fit_single_rslds

from rSLDS.plot_latents_footshock import load_and_plot_latents
from rSLDS.plot_latents_speed import plot_latents_speed_shocks
from rSLDS.plot_latents_pupil import plot_latents_pupil_shocks

from rSLDS.plot_discrete_states import plot_discrete_states          # style 1
from rSLDS.plot_discrete_states2 import plot_discrete_states2        # style 2

from rSLDS.mutual_information import latent_signal_mi, compare_mi_pre_post
from rSLDS.mi_pre_post import mi_pre_post_plot

from rSLDS.second_dimensionality_reduction import plot_rslds_vector_field
from rSLDS.forward_simulation_error import compute_and_plot_fse
from rSLDS.forward_simulation_error_observed import compute_and_plot_fse_observed_AR

from rSLDS.switch_statistics import switch_statistics, plot_switch_summary



## Plot Footshock With Latents

Plots latent dimensions with footshocks

### Compatible with the old model runs and the old data on drive

In [ ]:
RAT_IDS = [3, 4, 8, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21]
MODEL_SUFFIXES = [
    "ventral_responsive", "ventral",
    "dorsal_responsive", "dorsal",
    "responsive_all",
    "thalamus_responsive", "thalamus",
]

DEST_ROOT = Path("/content/drive/My Drive/rSLDS/All-Plots/latents_footshock")

for rid in RAT_IDS:
    rat_root = Path(f"/content/drive/My Drive/rSLDS/rSLDS-Model-Outputs/Rat{rid}-Model-Outputs")
    if not rat_root.exists():
        print(f"[Rat {rid}]missing folder.")
        continue
    for suffix in MODEL_SUFFIXES:
        model_sub = f"models_Rat{rid}_{suffix}"
        model_dir = rat_root / model_sub
        x_file = model_dir / "x_hat.npy"
        shock_file = model_dir / "footshock.npy"

        if not (x_file.exists() and shock_file.exists()):
            print(f"[Rat {rid}]  {suffix:20s} skipp.")
            continue

        print(f"[Rat {rid}]  {suffix:20s} → plotting…")

        try:
            fig_list = load_and_plot_latents(
                base_dir=str(rat_root),
                rat_tag="",
                model_sub=model_sub,
                integrate=True,
                smooth_sigma=3.0,
                colors=("k",),
                figsize=(10, 4),
                return_fig=True,
            )
        except Exception as e:
            print(f"no plotting  ({type(e).__name__}: {e})")
            traceback.print_exc(limit=1)
            continue

        out_dir = DEST_ROOT / f"Rat{rid}" / model_sub
        out_dir.mkdir(parents=True, exist_ok=True)

        for k, fig in enumerate(fig_list, start=1):
            for ext in ("png", "pdf"):  # save both formats
                fout = out_dir / f"latent{k:02d}.{ext}"
                fig.savefig(fout, dpi=300, bbox_inches="tight")
            plt.close(fig)
        print(f"saved {len(fig_list)}→ {out_dir}")

## Plot Latents With Speed

Plots latent dimensions with speed

### Compatible with the old model runs and the old data on drive

In [ ]:
RAT_IDS        = [3, 4, 8,10,11,13,14,15,16,17,18,19,20,21]
MODEL_SUFFIXES = [
    "ventral_responsive","ventral",
    "dorsal_responsive","dorsal",
    "responsive_all",
    "thalamus_responsive","thalamus",
]

DEST_ROOT = Path("/content/drive/My Drive/rSLDS/All-Plots/latents_speed")

for rid in RAT_IDS:
    rat_root = Path(f"/content/drive/My Drive/rSLDS/rSLDS-Model-Outputs/Rat{rid}-Model-Outputs")
    if not rat_root.exists():
        print(f"[Rat {rid}] missing folder.")
        continue

    for suffix in MODEL_SUFFIXES:
        model_sub  = f"models_Rat{rid}_{suffix}"
        model_dir  = rat_root / model_sub
        x_file     = model_dir / "x_hat.npy"
        shock_file = model_dir / "footshock.npy"
        speed_file = model_dir / "speed.npy"

        if not (x_file.exists() and shock_file.exists() and speed_file.exists()):
            continue

        print(f"[Rat {rid}]  {suffix:20s}plotting.")

        try:
            x_latents = np.load(x_file)
            footshock = np.load(shock_file)
            speed     = np.load(speed_file)

            # 10 ms bins
            duration  = len(speed) * 0.01
            figs = plot_latents_speed_shocks(
                x_latents               = x_latents,
                speed                   = speed,
                footshock_input         = footshock,
                duration                = duration,
                smooth_sigma_lat_s      = 0.10,   # 100 ms latent smoothing
                speed_smooth_sigma_s    = 0.20,   # 200 ms speed smoothing
                integrate_latent        = True,
                latent_color            = "black",
                speed_color             = "tab:blue",
                figsize                 = (10, 4),
                return_figs             = True,
            )
        except Exception as e:
            print(f" failed to save({type(e).__name__}: {e})")
            traceback.print_exc(limit=1)
            continue
        out_dir = DEST_ROOT / f"Rat{rid}" / model_sub
        out_dir.mkdir(parents=True, exist_ok=True)

        for k, fig in enumerate(figs, start=1):
            for ext in ("png", "pdf"):  # save both formats
                fout = out_dir / f"latent{k:02d}.{ext}"
                fig.savefig(fout, dpi=300, bbox_inches="tight")
            plt.close(fig)
        print(f"saved {len(figs)} figures in PNG and PDF {out_dir}")

## Plot Latents With Pupil

### Compatible with the old model runs and the old data on drive

In [ ]:
DRIVE = Path("/content/drive/My Drive")

MODEL_ROOT = DRIVE / "rSLDS/rSLDS-Model-Outputs"
HDF5_ROOT  = DRIVE / "rSLDS/Rat-Data-hdf5"
DEST_ROOT  = DRIVE / "rSLDS/All-Plots/latents_pupil"

H5_NAME = "NpxFiringRate_Behavior_SBL_10msBINS_0smoothing.hdf5"

RAT_IDS = [3,4,8,10,11,13,14,15,16,17,18,19,20,21]
MODEL_SUFFIXES = [
    "ventral_responsive","ventral",
    "dorsal_responsive","dorsal",
    "responsive_all",
    "thalamus_responsive","thalamus",
]

SMOOTH_SIGMA_S = 0.20    # seconds of smoothing for pupil
PCLIP = (1, 99)          # robust clip percentiles
FIGSIZE = (12, 4)

#helpers

def load_pupil_from_h5(h5):
    """
    Try several common keys (flat and nested) for the pupil signal.
    Returns (pupil_array, used_key or None).
    """
    candidates = [
        "pupil_diameter", "pupil", "pupil_size",
        "behavior/pupil_diameter", "behavior/pupil", "behavior/pupil_size"
    ]
    for key in candidates:
        if key in h5:
            return h5[key][...], key
    found = []
    def _visit(name, obj):
        if isinstance(obj, h5py.Dataset) and "pupil" in name.lower():
            found.append(name)
    h5.visititems(_visit)
    if found:
        return h5[found[0]][...], found[0]
    return None, None


def preprocess_pupil_raw(time_vec, pupil_vec, *, smooth_sigma_s=SMOOTH_SIGMA_S, pclip=PCLIP):
    """
    Smooth (optional) + robust clip to kill spikes, but **keep raw units**.
    No z-scoring, no normalization.
    """
    t = np.asarray(time_vec, float).ravel()
    p = np.asarray(pupil_vec, float).ravel()

    if smooth_sigma_s and smooth_sigma_s > 0:
        dt = float(t[1] - t[0]) if t.size > 1 else 1.0
        sigma = max(smooth_sigma_s / dt, 1e-9)
        p = gaussian_filter1d(p, sigma=sigma, mode="reflect")

    if pclip is not None:
        lo, hi = np.nanpercentile(p, pclip)
        if np.isfinite(lo) and np.isfinite(hi) and hi > lo:
            p = np.clip(p, lo, hi)

    return p


def build_footshock_mask(time_vec, shock_times):
    """
    Map shock timestamps onto the time grid using nearest-sample within ±½ dt.
    Robust against float mismatches (avoids np.isin on floats).
    """
    t = np.asarray(time_vec, float).ravel()
    st = np.asarray(shock_times, float).ravel()
    mask = np.zeros_like(t, dtype=bool)
    if t.size < 2 or st.size == 0:
        return mask
    idx = np.searchsorted(t, st, side="left")
    idx = np.clip(idx, 0, len(t)-1)
    dt = float(np.median(np.diff(t)))
    close = np.abs(t[idx] - st) <= (0.5 * dt)
    mask[idx[close]] = True
    return mask


def set_pupil_axis_nonnegative(figs):
    """Ensure the pupil (right) axis doesn't go below zero."""
    for fig in figs:
        for ax in fig.get_axes():
            label = (ax.get_ylabel() or "").lower()
            if "pupil" in label:
                ymin, ymax = ax.get_ylim()
                ax.set_ylim(bottom=0, top=ymax)

#main loop

plt.ioff()
total_saved = 0

for rid in RAT_IDS:
    rat_root = MODEL_ROOT / f"Rat{rid}-Model-Outputs"
    h5_path  = HDF5_ROOT  / f"Rat{rid}" / H5_NAME

    if not rat_root.exists():
        print(f"[Rat {rid}] model root missing — skipped")
        continue
    if not h5_path.exists():
        print(f"[Rat {rid}] HDF5 missing — {h5_path}")
        continue

    with h5py.File(h5_path, "r") as h5:
        time_vec    = h5["time"][...]
        shock_times = h5["footshock_times"][...]
        pupil_full, pupil_key = load_pupil_from_h5(h5)

    if pupil_full is None:
        print(f"[Rat {rid}] no pupil signal in HDF5 — skipped")
        continue
    else:
        print(f"[Rat {rid}] pupil source: {pupil_key}")

    for suffix in MODEL_SUFFIXES:
        model_dir = rat_root / f"models_Rat{rid}_{suffix}"
        x_path    = model_dir / "x_hat.npy"
        if not x_path.exists():
            continue

        try:
            x_latents = np.load(x_path)  # (T, D)
        except Exception as e:
            print(f"[Rat {rid}] {suffix:22s} failed to load x_hat ({type(e).__name__}: {e})")
            continue

        T_lat, D = x_latents.shape

        # match pupil & time to latents
        if len(pupil_full) != len(time_vec):
            pupil_full = pupil_full[:len(time_vec)]
        if len(pupil_full) != T_lat:
            f = interp1d(time_vec[:len(pupil_full)], pupil_full,
                         kind="linear", bounds_error=False, fill_value="extrapolate")
            time_use  = time_vec[:T_lat]
            pupil_use = f(time_use)
        else:
            time_use  = time_vec[:T_lat]
            pupil_use = pupil_full[:T_lat]

        # footshock mask on the same grid
        footshock_binary = build_footshock_mask(time_use, shock_times)

        # RAW pupil (smoothed + clipped only; keep units)
        pupil_proc = preprocess_pupil_raw(time_use, pupil_use)

        duration = float(time_use[-1])

        print(f"[Rat {rid}] {suffix:22s} → plotting (T={T_lat}, D={D})")

        try:
            figs = plot_latents_pupil_shocks(
                x_latents=x_latents,
                pupil_diameter=pupil_proc,
                footshock_input=footshock_binary,
                duration=duration,
                integrate_latent=True,
                smooth_sigma_lat_s=0.0,
                pupil_smooth_sigma_s=0.0,
                latent_color="black",
                pupil_color="tab:purple",
                figsize=FIGSIZE,
                return_figs=True,
            )
        except Exception as e:
            print(f"[Rat {rid}] {suffix:22s} plotting error ({type(e).__name__}: {e})")
            traceback.print_exc(limit=1)
            continue

        # keep pupil axis non-negative , we can changehere
        set_pupil_axis_nonnegative(figs)

        out_dir = DEST_ROOT / f"Rat{rid}" / f"models_Rat{rid}_{suffix}"
        out_dir.mkdir(parents=True, exist_ok=True)

        for k, fig in enumerate(figs, start=1):
            fig.savefig(out_dir / f"latent_vs_pupil_{k:02d}.png", dpi=300, bbox_inches="tight")
            plt.close(fig)
            total_saved += 1

        print(f"saved {len(figs)} PNG(s) → {out_dir}")

plt.ion()
print(f"\nAll done. Saved {total_saved} PNG(s) under:\n  {DEST_ROOT}")

This means that since we already have the old data and old runs and old runs outputs from the old models we can still keep them and test but we ll rerun the models with more discrete states so the new code is just for that change of directory.

## Plot Discrete States ( With 2 different styles for now )

### Compatible with the old model runs and the old data on drive

In [ ]:
RAT_IDS        = [3,4,8,10,11,13,14,15,16,17,18,19,20,21]
MODEL_SUFFIXES = [
    "ventral_responsive","ventral",
    "dorsal_responsive","dorsal",
    "responsive_all",
    "thalamus_responsive","thalamus",
]

MODEL_ROOT     = Path("/content/drive/My Drive/rSLDS/rSLDS-Model-Outputs")
CSV_AREA_ROOT  = Path("/content/drive/My Drive/rSLDS/Sub-Data/Seperate-by-Area")
CSV_RESP_ROOT  = Path("/content/drive/My Drive/rSLDS/Sub-Data/Only-Responsive")
DEST_ROOT      = Path("/content/drive/My Drive/rSLDS/All-Plots/discrete_states_both")

def area_subset_from_suffix(sfx: str):
    s = sfx.lower()
    # special-case the combined run
    if s == "responsive_all":
        return None, "responsive_all"
    area = "ventral"  if s.startswith("ventral")  else \
           "dorsal"   if s.startswith("dorsal")   else \
           "thalamus" if s.startswith("thalamus") else None
    if area is None:
        raise ValueError(f"Bad suffix: {sfx}")
    if   "responsive" in s: subset = "responsive"
    else:                    subset = "allActive"
    return area, subset

plt.ioff()

for rid in RAT_IDS:
    rat_root = MODEL_ROOT / f"Rat{rid}-Model-Outputs"
    if not rat_root.exists():
        print(f"[Rat {rid}] root folder missing — skipped")
        continue

    for suffix in MODEL_SUFFIXES:
        model_sub = f"models_Rat{rid}_{suffix}"
        model_dir = rat_root / model_sub

        z_file     = model_dir / "z_hat.npy"
        shock_file = model_dir / "footshock.npy"
        if not (z_file.exists() and shock_file.exists()):
            continue

        try:
            AREA, SUBSET = area_subset_from_suffix(suffix)
        except ValueError:
            print(f"[Rat {rid}] {suffix:22s}  bad suffix — skipped")
            continue

        # pick the correct CSV path
        if AREA is None and SUBSET == "responsive_all":
            csv_path = (CSV_RESP_ROOT / f"Rat{rid}" /
                        f"area_splits_rat{rid}_responsive" / "responsive_rates_raw.csv")
        else:
            csv_path = (CSV_AREA_ROOT / f"Rat{rid}" /
                        f"area_splits_rat{rid}_{SUBSET}" / f"{AREA}_wide.csv")

        if not csv_path.exists():
            print(f"[Rat {rid}] {suffix:22s}  CSV missing — skipped ({csv_path})")
            continue

        # load
        try:
            z_states      = np.load(z_file)
            footshock_vec = np.load(shock_file)
            time_vec      = pd.read_csv(csv_path, usecols=["time_s"])["time_s"].values
        except Exception as e:
            print(f"[Rat {rid}] {suffix:22s}  load error: {e}")
            traceback.print_exc(limit=1)
            continue

        if not (len(z_states) == len(time_vec) == len(footshock_vec)):
            print(f"[Rat {rid}] {suffix:22s}  length mismatch — skipped")
            continue

        out_dir = DEST_ROOT / f"Rat{rid}" / model_sub
        out_dir.mkdir(parents=True, exist_ok=True)

        title_area = "responsive_all" if AREA is None else AREA

        # style 1
        try:
            fig1, _ = plot_discrete_states(
                z_states      = z_states,
                time_vec      = time_vec,
                shock_times   = footshock_vec,
                palette       = "Set1",
                lw            = 6,
                min_duration  = 0.5,
                dot_style     = "line",
                title         = f"Rat {rid} – {title_area} – discrete states (style 1)",
                figsize       = (12, 2.5),
            )
            fig1.savefig(out_dir / "states_style1.png", dpi=300, bbox_inches="tight")
            plt.close(fig1)
        except Exception as e:
            print(f"[Rat {rid}] {suffix:22s}  style1 failed: {e}")
            traceback.print_exc(limit=1)

        # style 2
        try:
            plot_discrete_states2(
                z_states      = z_states,
                time_vec      = time_vec,
                shock_times   = footshock_vec,
                palette       = "Set1",
                lw            = 6,
                title         = f"Rat {rid} – {title_area} – discrete states (style 2)",
                figsize       = (12, 2.5),
            )
            fig2 = plt.gcf()
            fig2.savefig(out_dir / "states_style2.png", dpi=300, bbox_inches="tight")
            plt.close(fig2)
        except Exception as e:
            print(f"[Rat {rid}] {suffix:22s}  style2 failed: {e}")
            traceback.print_exc(limit=1)

        print(f"[Rat {rid}] {suffix:22s} saved → {out_dir}")

plt.ion()


## Switch Statistics

Shows discrete state switching statitsics for pre shock and after shock observation. How the states changes between each other happen and its statistics.

### Compatible with the old model runs and the old data on drive

In [ ]:
RAT_IDS        = [3, 4, 8, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21]
MODEL_SUFFIXES = [
    "ventral_responsive","ventral",
    "dorsal_responsive","dorsal",
    "responsive_all",
    "thalamus_responsive","thalamus",
]

MODEL_ROOT     = Path("/content/drive/My Drive/rSLDS/rSLDS-Model-Outputs")
CSV_AREA_ROOT  = Path("/content/drive/My Drive/rSLDS/Sub-Data/Seperate-by-Area")
CSV_RESP_ROOT  = Path("/content/drive/My Drive/rSLDS/Sub-Data/Only-Responsive")
DEST_ROOT      = Path("/content/drive/My Drive/rSLDS/All-Plots/switch_summary")

def area_subset_from_suffix(sfx: str):
    s = sfx.lower()
    if s == "responsive_all":
        return None, "responsive_all"
    area = "ventral"  if s.startswith("ventral")  else \
           "dorsal"   if s.startswith("dorsal")   else \
           "thalamus" if s.startswith("thalamus") else None
    if area is None:
        raise ValueError(f"cannot infer area from suffix {sfx!r}")
    subset = "responsive" if "responsive" in s else "allActive"
    return area, subset

plt.ioff()

for rid in RAT_IDS:
    rat_root = MODEL_ROOT / f"Rat{rid}-Model-Outputs"
    if not rat_root.exists():
        print(f"[Rat {rid:2d}] root folder missing – skipped.")
        continue

    for suffix in MODEL_SUFFIXES:
        model_sub  = f"models_Rat{rid}_{suffix}"
        model_dir  = rat_root / model_sub
        z_file     = model_dir / "z_hat.npy"
        shock_file = model_dir / "footshock.npy"

        if not (z_file.exists() and shock_file.exists()):
            continue

        try:
            AREA, SUBSET = area_subset_from_suffix(suffix)
        except ValueError:
            print(f"[Rat {rid:2d}] {suffix:24s} – bad suffix")
            continue

        # pick the correct CSV path
        if AREA is None and SUBSET == "responsive_all":
            csv_path = (CSV_RESP_ROOT / f"Rat{rid}" /
                        f"area_splits_rat{rid}_responsive" / "responsive_rates_raw.csv")
        else:
            csv_path = (CSV_AREA_ROOT / f"Rat{rid}" /
                        f"area_splits_rat{rid}_{SUBSET}" / f"{AREA}_wide.csv")

        if not csv_path.exists():
            print(f"[Rat {rid:2d}] {suffix:24s} – CSV missing: {csv_path}")
            continue

        # load
        try:
            z_states      = np.load(z_file).astype(int).ravel()
            footshock_vec = np.load(shock_file).squeeze()  # could be (T,1) or (T,)
            df            = pd.read_csv(csv_path)
            # prefer 'time_s' but fall back to the first time-like column if needed
            time_col = "time_s" if "time_s" in df.columns else df.columns[0]
            time_vec      = df[time_col].to_numpy()
        except Exception as e:
            print(f"[Rat {rid:2d}] {suffix:24s} – load error ({e})")
            traceback.print_exc(limit=1)
            continue

        if not (len(z_states)==len(time_vec)==len(footshock_vec)):
            print(f"[Rat {rid:2d}] {suffix:24s} – length mismatch "
                  f"(z={len(z_states)}, t={len(time_vec)}, shock={len(footshock_vec)})")
            continue

        out_dir = DEST_ROOT / f"Rat{rid}" / model_sub
        out_dir.mkdir(parents=True, exist_ok=True)

        # run summary plots
        try:
            stats, figs = plot_switch_summary(
                z_states       = z_states,
                time_vec       = time_vec,
                footshock_mask = footshock_vec.astype(bool),
                separate       = True,   # returns 3 figs
                dpi            = 150,
            )
        except Exception as e:
            print(f"[Rat {rid:2d}] {suffix:24s} – plot error ({e})")
            traceback.print_exc(limit=1)
            continue

        # save figures
        label_map = ["switch_count", "switch_rate", "switch_occupancy"]
        for tag, fig in zip(label_map, figs):
            fig.savefig(out_dir / f"{tag}.png", dpi=300, bbox_inches="tight")
            plt.close(fig)

        # optionally save stats as CSV if it's a dict/DataFrame-like
        try:
            import pandas as pd
            if hasattr(stats, "to_csv"):
                stats.to_csv(out_dir / "switch_stats.csv", index=False)
            elif isinstance(stats, dict):
                pd.DataFrame([stats]).to_csv(out_dir / "switch_stats.csv", index=False)
        except Exception:
            pass

        print(f"[Rat {rid:2d}] {suffix:24s} ✓  saved in {out_dir}")

plt.ion()


## Mutual Information Latents / Pupil/ Speed

Mutual Information Criterion between latent dimensions and speed. The amount of information one random variable contains about another. How much knowing one variable reduces the uncertainty about the oter variable. Related to the concept entropy which is a measure of uncertainty or randomness in a single random variable. Mutual Information (MI): The difference between the entropy of one variable and its conditional entropy given another variable. It quantifies how much knowing one variable reduces the uncertainty of the other.

### Compatible with the old model runs and the old data on drive

**How to read the output?**
dim → which latent dimension.

MI_raw → how much info (mutual information) that latent has with speed.

thr95 → the “95% cutoff” from shuffled data (the null).

p → how often shuffles were as strong as the real data (a p-value).

Red star = significant

Different shuffle types exist:

circular → shift the signal in time,

strict → scramble phases but keep spectrum,

permute → fully random.

we can chose which one with SHUFFLE_METHOD, and how many times with N_SHUFFLE.
here i m shuffling the behavioural signals.


In [ ]:
from pathlib import Path
import numpy as np, pandas as pd, matplotlib.pyplot as plt, traceback

# make sure latent_signal_mi is available:
from mutual_information import latent_signal_mi

RAT_IDS        = [3, 4, 8, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21]
MODEL_SUFFIXES = [
    "ventral_responsive","ventral",
    "dorsal_responsive","dorsal",
    "responsive_all",
    "thalamus_responsive","thalamus",
]

MODEL_ROOT     = Path("/content/drive/My Drive/rSLDS/rSLDS-Model-Outputs")
CSV_AREA_ROOT  = Path("/content/drive/My Drive/rSLDS/Sub-Data/Seperate-by-Area")
CSV_RESP_ROOT  = Path("/content/drive/My Drive/rSLDS/Sub-Data/Only-Responsive")
DEST_BASE      = Path("/content/drive/My Drive/rSLDS/All-Plots")

WIN_S             = 0.5
INTEGRATE_LATENTS = False
SHUFFLE_METHOD    = "permute"      # "circular"|"strict"|"permute"
N_SHUFFLE         = 1000
FDR_ALPHA         = 0.05
RNG_SEED          = 0

# helpers
def area_subset_from_suffix(sfx: str):
    s = sfx.lower()
    if s == "responsive_all":
        return None, "responsive_all"
    area = "ventral"  if s.startswith("ventral")  else \
           "dorsal"   if s.startswith("dorsal")   else \
           "thalamus" if s.startswith("thalamus") else None
    if area is None:
        raise ValueError(f"cannot infer area from suffix {sfx!r}")
    subset = "responsive" if "responsive" in s else "allActive"
    return area, subset

def get_pupil_from_csv(df: pd.DataFrame):
    for col in ("pupil", "pupil_diameter", "pupil_size"):
        if col in df.columns:
            return df[col].to_numpy()
    return None

def get_speed_from_csv(df: pd.DataFrame):
    if "speed" in df.columns:
        return df["speed"].to_numpy()
    for cand in ("velocity","speed_cm_s","beh_speed"):
        if cand in df.columns:
            return df[cand].to_numpy()
    return None


for rid in RAT_IDS:
    rat_root = MODEL_ROOT / f"Rat{rid}-Model-Outputs"
    if not rat_root.exists():
        print(f"[Rat {rid:2d}] root folder missing")
        continue

    for suffix in MODEL_SUFFIXES:
        model_sub = f"models_Rat{rid}_{suffix}"
        model_dir = rat_root / model_sub

        x_file    = model_dir / "x_hat.npy"
        speed_npy = model_dir / "speed.npy"
        pupil_npy = model_dir / "pupil.npy"

        # figure out which CSV to use
        try:
            AREA, SUBSET = area_subset_from_suffix(suffix)
        except ValueError:
            continue

        if AREA is None and SUBSET == "responsive_all":
            csv_path = (CSV_RESP_ROOT / f"Rat{rid}" /
                        f"area_splits_rat{rid}_responsive" / "responsive_rates_raw.csv")
        else:
            csv_path = (CSV_AREA_ROOT / f"Rat{rid}" /
                        f"area_splits_rat{rid}_{SUBSET}" / f"{AREA}_wide.csv")

        if not (x_file.exists() and csv_path.exists()):
            continue

        try:
            x_latents = np.load(x_file)
            df_csv    = pd.read_csv(csv_path)

            # time column
            time_col  = "time_s" if "time_s" in df_csv.columns else df_csv.columns[0]
            time_vec  = df_csv[time_col].to_numpy()

            # speed
            if speed_npy.exists():
                speed = np.load(speed_npy)
            else:
                speed = get_speed_from_csv(df_csv)

            # pupil
            if pupil_npy.exists():
                pupil = np.load(pupil_npy)
            else:
                pupil = get_pupil_from_csv(df_csv)

        except Exception as e:
            print(f"[Rat {rid:2d}] {suffix:24s} – load error ({e})")
            traceback.print_exc(limit=1)
            continue

        # pack available signals
        signals = {}
        if speed is not None: signals["speed"] = speed
        if pupil is not None: signals["pupil"] = pupil
        if not signals:
            print(f"[Rat {rid:2d}] {suffix:24s} – no speed/pupil found")
            continue

        for sig_name, sig in signals.items():
            if not (len(x_latents) == len(sig) == len(time_vec)):
                print(f"[Rat {rid:2d}] {suffix:24s} – length mismatch ({sig_name})")
                continue

            sig_clean = np.nan_to_num(sig, nan=np.nanmean(sig))

            # === MI with rich output (includes null mean, MI_delta, p_perm, FDR) ===
            mi_rec = latent_signal_mi(
                latents           = x_latents,
                signal            = sig_clean,
                time_vec          = time_vec,
                win_s             = WIN_S,
                integrate_latents = INTEGRATE_LATENTS,
                shuffle           = SHUFFLE_METHOD,
                n_shuffle         = N_SHUFFLE,
                random_state      = RNG_SEED,
                return_rich       = True,
                fdr_alpha         = FDR_ALPHA,
            )

            df_mi = pd.DataFrame.from_records(mi_rec)

            # add metadata about the run so it's traceable
            df_mi["shuffle"]           = SHUFFLE_METHOD
            df_mi["n_shuffle"]         = N_SHUFFLE
            df_mi["win_s"]             = WIN_S
            df_mi["integrate_latents"] = INTEGRATE_LATENTS
            df_mi["rat"]               = rid
            df_mi["model_sub"]         = model_sub
            df_mi["signal"]            = sig_name

            # where to save
            DEST_ROOT = DEST_BASE / f"mi_{sig_name}"
            out_dir   = DEST_ROOT / f"Rat{rid}" / model_sub
            out_dir.mkdir(parents=True, exist_ok=True)

            # save CSV (now includes MI_null_mean and MI_delta = MI_raw - MI_null_mean)
            df_mi.to_csv(out_dir / f"mi_latents_{sig_name}.csv", index=False)

            dims = df_mi["dim"].to_numpy()
            mi_vals = df_mi["MI_raw"].to_numpy()

            fig, ax = plt.subplots(figsize=(0.65*len(dims)+3, 3))
            ax.bar(dims, mi_vals, color="tab:blue", alpha=.85)

            # prefer FDR decision if present; else p_perm<.05; else nothing
            star_mask = None
            star_label = None
            if "reject_bh" in df_mi.columns and df_mi["reject_bh"].any():
                star_mask  = df_mi["reject_bh"].to_numpy().astype(bool)
                star_label = f"FDR q ≤ {FDR_ALPHA:g}"
            elif "p_perm" in df_mi.columns:
                star_mask  = (df_mi["p_perm"].to_numpy() < 0.05)
                star_label = "p_perm < .05"

            if star_mask is not None and star_mask.any():
                offset = 0.02 * (mi_vals.max() if np.isfinite(mi_vals.max()) and mi_vals.max() > 0 else 1.0)
                ax.scatter(dims[star_mask], (mi_vals + offset)[star_mask],
                           marker="*", color="red", zorder=3, label=star_label)
                ax.legend(frameon=False)

            ax.set_xlabel("latent dim")
            ax.set_ylabel("MI (bits)")
            ax.set_title(f"Rat {rid} – {suffix}  ({sig_name} vs. latents)\n"
                         f"ΔMI = MI_raw − mean(null), shuffle={SHUFFLE_METHOD}, n={N_SHUFFLE}")
            fig.tight_layout()
            fig.savefig(out_dir / f"mi_bar_{sig_name}.png", dpi=300, bbox_inches="tight")
            plt.close(fig)

            print(f"[Rat {rid:2d}] {suffix:24s} ✓  MI({sig_name}) → {out_dir}")


dim – latent index 0 = first latent.

MI_raw – MI between this latent and the behavior on real (unshuffled) data.
Example: 0.0316.

MI_null_mean / MI_null_std – average and SD of MI under the null.
Example: null mean 0.00547, SD 0.00800.

MI_delta – how much larger the real MI is than the null mean: MI_raw − MI_null_mean.


thr95 – the 95th percentile of the null; a simple “uncorrected” threshold.


z – z-score: (MI_raw − MI_null_mean) / MI_null_std.

p_perm – permutation p-value: fraction of null ≥ real (with +1 correction).

significant_95 – TRUE if MI_raw > thr95 (uncorrected, per-dim).

q_bh / reject_bh – Benjamini–Hochberg FDR across dims; reject_bh says whether it’s significant after multiple-comparisons at your FDR_ALPHA (0.05).

shuffle, n_shuffle – how the null was built (permute) and how many surrogates (1000).

win_s, integrate_latents – MI computed on 0.5 s windows; FALSE means raw latents (not integrated).

rat, model_sub, signal – provenance (which animal, run folder, and behavior).

## MI Pre-Post Shock

In order to see the mutual information  pre /post shock observation with the external inputs pupil and speed.

### Compatible with the old model runs and the old data on drive

**How to read it ?**

Blue bar = MI(latent d, behaviour) before the first shock.

Red bar = MI after the first shock.

Red star (★) on a red bar = that post-shock MI is significant vs the shuffled-null (either p < .05 or above the null’s 95th percentile, depending on the script title).

Higher bar ⇒ stronger relationship (in bits) between that latent and the behaviour in that segment.
dim – latent index.

MI_raw – the measured MI (bits) between that latent (windowed) and the behaviour.

MI_min – minimum MI from the shuffled null (mostly for reference; you can ignore).

thr95 – the 95th percentile of the null MI (the “significance threshold”).

p – permutation p-value: fraction of shuffled MI ≥ MI_raw (smaller = more significant).

In [ ]:
from pathlib import Path
import numpy as np, pandas as pd, matplotlib.pyplot as plt, traceback, h5py

RAT_IDS        = [3,4,8,10,11,13,14,15,16,17,18,19,20,21]
MODEL_SUFFIXES = [
    "ventral_responsive","ventral",
    "dorsal_responsive","dorsal",
    "responsive_all",
    "thalamus_responsive","thalamus",
]

MODEL_ROOT = Path("/content/drive/My Drive/rSLDS/rSLDS-Model-Outputs")
CSV_AREA_ROOT = Path("/content/drive/My Drive/rSLDS/Sub-Data/Seperate-by-Area")
CSV_RESP_ROOT = Path("/content/drive/My Drive/rSLDS/Sub-Data/Only-Responsive")
H5_ROOT   = Path("/content/drive/My Drive/rSLDS/Rat-Data-hdf5")
DEST_ROOT = Path("/content/drive/My Drive/rSLDS/All-Plots/mi_prepost")

def area_subset_from_suffix(sfx: str):
    s = sfx.lower()
    if s == "responsive_all":
        return None, "responsive_all"
    area = "ventral"  if s.startswith("ventral")  else \
           "dorsal"   if s.startswith("dorsal")   else \
           "thalamus" if s.startswith("thalamus") else None
    if area is None:
        raise ValueError(f"Bad suffix: {sfx}")
    subset = "responsive" if "responsive" in s else "allActive"
    return area, subset

def normalize_footshock_mask(time_vec: np.ndarray, footshock) -> np.ndarray:
    t  = np.asarray(time_vec, float).ravel()
    fs = np.asarray(footshock).ravel()
    T  = len(t); mask = np.zeros(T, dtype=bool)
    if fs.size == T: return fs.astype(bool)
    if np.issubdtype(fs.dtype, np.integer):
        idx = fs[(fs >= 0) & (fs < T)].astype(int); mask[idx] = True; return mask
    ts = fs.astype(float); ts = ts[(ts >= t[0]) & (ts <= t[-1])]
    if ts.size == 0: return mask
    idx = np.searchsorted(t, ts, side="left"); idx = np.clip(idx, 0, T-1)
    dt = float(np.median(np.diff(t))); close = np.abs(t[idx] - ts) <= (0.5 * dt)
    if close.size == idx.size: idx = idx[close]
    mask[idx] = True; return mask

def load_signal_h5_or_csv(rid: int, df: pd.DataFrame, key_candidates_h5, key_candidates_csv):
    """Try HDF5 first (preferred), then CSV. Returns np.ndarray or None."""
    h5_path = H5_ROOT / f"Rat{rid}" / "NpxFiringRate_Behavior_SBL_10msBINS_0smoothing.hdf5"
    if h5_path.exists():
        try:
            with h5py.File(h5_path, "r") as h5:
                for k in key_candidates_h5:
                    if k in h5:
                        return np.asarray(h5[k][...]).squeeze()
        except Exception as e:
            print(f"[Rat {rid}] HDF5 read error ({e})")
    for c in key_candidates_csv:
        if c in df.columns:
            return df[c].to_numpy()
    return None

for rid in RAT_IDS:
    rat_root = MODEL_ROOT / f"Rat{rid}-Model-Outputs"
    if not rat_root.exists():
        print(f"[Rat {rid}]   couldnt ")
        continue

    for suffix in MODEL_SUFFIXES:
        model_sub  = f"models_Rat{rid}_{suffix}"
        model_dir  = rat_root / model_sub

        x_file     = model_dir / "x_hat.npy"
        shock_file = model_dir / "footshock.npy"
        if not (x_file.exists() and shock_file.exists()):
            continue

        try:
            AREA, SUBSET = area_subset_from_suffix(suffix)
        except ValueError:
            print(f"[Rat {rid}] {suffix:22s}  bad suffix – skipped")
            continue

        # choose the correct CSV path
        if AREA is None and SUBSET == "responsive_all":
            csv_path = (CSV_RESP_ROOT / f"Rat{rid}" /
                        f"area_splits_rat{rid}_responsive" / "responsive_rates_raw.csv")
        else:
            csv_path = (CSV_AREA_ROOT / f"Rat{rid}" /
                        f"area_splits_rat{rid}_{SUBSET}" / f"{AREA}_wide.csv")
        if not csv_path.exists():
            print(f"[Rat {rid}] {suffix:22s}  CSV missing – skipped")
            continue

        # load shared
        try:
            latents       = np.load(x_file)               # (T,D)
            footshock_vec = np.load(shock_file)           # mask/idx/ts
            df            = pd.read_csv(csv_path)
            time_col      = "time_s" if "time_s" in df.columns else df.columns[0]
            time_vec      = df[time_col].to_numpy()

            speed = load_signal_h5_or_csv(
                rid, df,
                key_candidates_h5=["speed", "behavior/speed"],
                key_candidates_csv=["speed", "velocity", "speed_cm_s", "beh_speed"]
            )
            pupil = load_signal_h5_or_csv(
                rid, df,
                key_candidates_h5=["pupil", "pupil_diameter", "behavior/pupil"],
                key_candidates_csv=["pupil", "pupil_diameter", "pupil_size"]
            )
        except Exception as e:
            print(f"[Rat {rid}] {suffix:22s}  load error ({e})")
            traceback.print_exc(limit=1)
            continue

        fs_mask = normalize_footshock_mask(time_vec, footshock_vec)
        if not fs_mask.any():
            print(f"[Rat {rid}] {suffix:22s}  (no shock found) – skipped")
            continue

        signals = {"speed": speed, "pupil": pupil}
        for sig_name, sig in signals.items():
            if sig is None:
                print(f"[Rat {rid}] {suffix:22s}  no {sig_name} in HDF5/CSV – skipped")
                continue
            if not (len(latents) == len(sig) == len(time_vec)):
                print(f"[Rat {rid}] {suffix:22s}  length mismatch ({sig_name}) – skipped")
                continue

            sig_clean = np.nan_to_num(sig, nan=float(np.nanmean(sig)))

            print(f"[Rat {rid}] {suffix:22s}  → MI pre/post ({sig_name}) …")
            try:
                table, fig = mi_pre_post_plot(
                    latents           = latents,
                    signal            = sig_clean,
                    time_vec          = time_vec,
                    footshock_mask    = fs_mask,
                    win_s             = 0.3,
                    integrate_latents = False,
                    shuffle           = "permute",
                    n_shuffle         = 1000,
                    color_pre         = "#38588C",
                    color_post        = "#D73027",
                )
            except Exception as e:
                print(f" MI failed ({sig_name}) ({e.__class__.__name__}: {e})")
                traceback.print_exc(limit=1)
                continue

            out_dir = DEST_ROOT / f"Rat{rid}" / model_sub
            out_dir.mkdir(parents=True, exist_ok=True)
            table.to_csv(out_dir / f"MI_pre_post_{sig_name}.csv", index=False)
            fig.savefig(out_dir / f"MI_pre_post_{sig_name}.png", dpi=300, bbox_inches="tight")
            plt.close(fig)
            print(f"saved → {out_dir} ({sig_name})")


## State Vector Field

Each rSLDS state has its own dynamics model (rules for how latent activity changes over time).

A vector field is a map of arrows in latent space:

Each arrow shows the direction and speed of change in neural activity if the system is at that point.

This lets us see how brain activity evolves differently across states:

Some states may stabilize activity (arrows pointing inward).

Others may create rotations or oscillations (arrows circling around).

In short, vector fields visualize the “flow of neural population activity” under each discrete state.

### Compatible with the old model runs and the old data on drive

In [ ]:
RAT_IDS        = [3,4,8,10,11,13,14,15,16,17,18,19,20,21]
MODEL_SUFFIXES = [
    "ventral_responsive","ventral",
    "dorsal_responsive","dorsal",
    "responsive_all",
    "thalamus_responsive","thalamus",
]

MODEL_ROOT = Path("/content/drive/My Drive/rSLDS/rSLDS-Model-Outputs")
DEST_ROOT  = Path("/content/drive/My Drive/rSLDS/All-Plots/vector_fields")

def has_needed_files(model_dir: Path) -> bool:
    return (
        glob.glob(str(model_dir / "x_hat*.npy")) and
        glob.glob(str(model_dir / "z_hat*.npy")) and
        glob.glob(str(model_dir / "*model*.pkl"))
    )

plt.ioff()

for rid in RAT_IDS:
    rat_root = MODEL_ROOT / f"Rat{rid}-Model-Outputs"
    if not rat_root.exists():
        print(f"[Rat {rid}]root folder missing")
        continue

    for suffix in MODEL_SUFFIXES:
        model_sub = f"models_Rat{rid}_{suffix}"
        model_dir = rat_root / model_sub

        if not has_needed_files(model_dir):
            continue

        try:
            print(f"[Rat {rid}] {suffix:22s} → vector field …")
            plot_rslds_vector_field(
                model_dir = str(model_dir),
                grid_size = 30,
                cmap      = "plasma",
                density   = 1.0,
            )

            fig = plt.gcf()
            out_dir = DEST_ROOT / f"Rat{rid}" / model_sub
            out_dir.mkdir(parents=True, exist_ok=True)
            fig.savefig(out_dir / "vector_field.png", dpi=300, bbox_inches="tight")
            plt.close(fig)

            print(f"saved → {out_dir}/vector_field.png")

        except Exception as e:
            print(f" couldnt ({e.__class__.__name__}: {e})")
            traceback.print_exc(limit=1)
            continue
plt.ion()


## Forward Simulation Error (FSE)

When simulating the rSLDS forward in time, the model predicts the next state based on its current state and the active linear system. The forward simulation error is then calculated by comparing this predicted state with the actual observed state at the next time step. A common metric for this is the mean squared error (MSE) or a normalized version of it. We used Forward Simulation Error (FSE) to test how well the rSLDS captured latent dynamics. So I  used Forward Simulation Error (FSE) to test how well the rSLDS captured latent dynamics. Starting from the inferred latent state xt, rolled the model forward for Δt steps and compared predictions to the posterior mean at 𝑥𝑡+Δ𝑡.
If you see where the rat is right now, you can guess pretty well where it will be one step later (small FSE).

If you try to guess 10 steps later without looking again, your guess will drift off — because the rat might turn, speed up, or freeze (big FSE).

How fast the model's predictions drift away from reality.

In [ ]:
RAT_IDS        = [3,4,8,10,11,13,14,15,16,17,18,19,20,21]
MODEL_SUFFIXES = [
    "ventral_responsive","ventral",
    "dorsal_responsive","dorsal",
    "responsive_all",
    "thalamus_responsive","thalamus",
]

MODEL_ROOT = Path("/content/drive/My Drive/rSLDS/rSLDS-Model-Outputs")
DEST_ROOT  = Path("/content/drive/My Drive/rSLDS/All-Plots/fse")

def find_model_fname(model_dir: Path) -> str | None:
    for name in ("my_model.pkl", "rSLDS.pkl"):
        if (model_dir / name).exists():
            return name
    for pat in ("*model*.pkl", "*.pkl"):
        hits = glob.glob(str(model_dir / pat))
        if hits:
            return Path(hits[0]).name
    return None

plt.ioff()

for rid in RAT_IDS:
    rat_root = MODEL_ROOT / f"Rat{rid}-Model-Outputs"
    if not rat_root.exists():
        print(f"[Rat {rid}] root folder missing — skipped")
        continue

    for suffix in MODEL_SUFFIXES:
        model_sub = f"models_Rat{rid}_{suffix}"
        model_dir = rat_root / model_sub

        x_path, z_path = model_dir / "x_hat.npy", model_dir / "z_hat.npy"
        if not (x_path.exists() and z_path.exists()):
            continue

        model_fname = find_model_fname(model_dir)
        if model_fname is None:
            continue

        try:
            print(f"[Rat {rid}] {suffix:22s} → FSE …")
            fse_vals = compute_and_plot_fse(
                model_dir   = model_dir,
                n_steps     = 10,
                plot        = False,
                model_fname = model_fname,
            )

            fig, ax = plt.subplots(figsize=(5, 3))
            x = np.arange(1, len(fse_vals) + 1)
            ax.plot(x, fse_vals, marker='o')
            ax.set_xlabel('Δt steps ahead')
            ax.set_ylabel('Forward Simulation MSE (latents)')
            ax.set_title('Forward Simulation Error (FSE)')
            ax.spines[['top','right']].set_visible(False)
            fig.tight_layout()

            out_dir = DEST_ROOT / f"Rat{rid}" / model_sub
            out_dir.mkdir(parents=True, exist_ok=True)
            fig.savefig(out_dir / "FSE.png", dpi=300, bbox_inches="tight")
            np.save(out_dir / "FSE.npy", np.asarray(fse_vals, float))
            plt.close(fig)

            print(f"saved → {out_dir}")

        except Exception as e:
            print(f"FSE failed ({e.__class__.__name__}: {e})")
            traceback.print_exc(limit=1)
            continue

plt.ion()
